In [338]:
import requests
from bs4 import BeautifulSoup

## Functions:

In [328]:
def get_soup(url):
    """This function returns Soup object from given url"""
    resp = requests.get(url)
    if resp.status_code == 404:
        raise NameError('incorrect path')
    return BeautifulSoup(resp.content)

In [329]:
def get_user_info(username):
    """This function receives user_name of a GitHub account.
    Return the dictionary with following keys respectively to GH-account:
    -user_name
    -organization
    -city
    -followers
    -repositories"""
    userinfo = {}
    user = 'https://github.com/' + username
    soup = get_soup(user)
    user_name = soup.find('span', class_="p-name vcard-fullname d-block overflow-hidden").text.strip()
    if not user_name:
        userinfo['user_name'] = 'unknown'
    else:        
        userinfo['user_name'] = user_name
    try:
        org_name = soup.find("span", class_="p-org").text
        userinfo['organization'] = org_name
    except AttributeError:
        userinfo['organization'] = 'unknown'
    try:
        city = soup.find('span',class_="p-label").text
        userinfo['city'] = city
    except AttributeError:
        userinfo['city'] = 'unknown'
    try:
        followers = soup.find("a", class_='Link--secondary no-underline no-wrap').find(
            "span", class_='text-bold color-fg-default').text
        userinfo['followers'] = followers
    except AttributeError:
        userinfo['followers'] = 0
    try:
        repo = soup.find('nav', class_='UnderlineNav-body width-full p-responsive').find(
            'span', class_='Counter').text
        userinfo['repositories'] = repo
    except AttributeError:
        userinfo['repositories'] = 0
    
    return userinfo
        

In [330]:
def get_user_repositories(username):
    """It receives username and returns the dictionary with all public repositories
    of a given username (GH-account) and used languages of these repositories"""
    repos = {}
    user = 'https://github.com/' + username + '?tab=repositories'
    soup = get_soup(user)
    user_name = soup.find('span', class_="p-name vcard-fullname d-block overflow-hidden").text.strip()
    if not user_name:
        repos['username'] = 'Unknown user name of ' + 'https://github.com/' + username
    else: 
        repos['username'] = user_name
    found_repo = soup.find(
        'div', id="user-repositories-list").find_all("div", class_='col-10 col-lg-9 d-inline-block')
    for repo in found_repo:
        if repo.find("span", class_='Label Label--secondary v-align-middle ml-1 mb-1').text.strip() == 'Public':
            try:
                lang = repo.find('span', itemprop='programmingLanguage').text
                repos[repo.find("a").text.strip()] = lang.strip()
            except AttributeError:
                repos[repo.find("a").text.strip()] = 'None'
    
    return repos

In [333]:
def list_repository_contents(username, repository):
    """Returns repository content from given account, repository and repository_path"""
    content = []
    # Не понимаю, зачем нужен аргумент repository_path. Это же полный путь до репозитория.
    # Так что оставлю username and repository (можно наоборот)
    github_path = 'https://github.com/' + username + '/' + repository
    soup = get_soup(github_path)
    grid = soup.find('div', role='grid').find_all('div', role='row')
    for element in grid:
        header = element.find('div', role='rowheader')
        try:
            content.append(header.find('a').text)
        except AttributeError:
            continue
    return content

In [462]:
def download_file(remote_file_path, local_file_path):
    """Download file with recieved remote path to local path"""
    # Также и здесь достаточно ведь remo_file_path заменяет и username и repository
    # e.g. : https://github.com/keyreallkeyrealenko/BI_2021_Python/blob/main/.gitignore - remote_file_path
    # удалю здесь username и repository
    # Здесь очень костыли, совсем не успевал
    soup = get_soup(remote_file_path)
    groups = soup.find_all('div',class_='BtnGroup')
    link = ''
    for group in groups:
        if group.find('a', class_='js-permalink-replaceable-link btn-sm btn BtnGroup-item'):
            link = group.find('a', class_='js-permalink-replaceable-link btn-sm btn BtnGroup-item')
    if not link:
        raise ValueError('you are trying to download a full directory')
    url = 'https://raw.githubusercontent.com' + link.get('href').replace('raw/', '') 
    req = requests.get(url,allow_redirects=True)
    with open(local_file_path, 'wb') as file:
        file.write(req.content)

In [464]:
download_file('https://github.com/eriklindernoren/PyTorch-YOLOv3/blob/master/.github/dependabot.yml', './file')

In [457]:
if not a:
    print(2)

NameError: name 'a' is not defined

In [337]:
list_repository_contents('krglkvrmn', 'IB_Music_project')

['ANN_genre_classification',
 'clustering',
 'data',
 'feature_extraction_and_data_preprocessing',
 'random_forest_genre_classification',
 'segmentation',
 'README.md',
 'requirements.txt']

In [288]:
get_user_repositories('krglkvrmn')

{'username': 'Roman Kruglikov',
 'FlowSOM': 'Python',
 'Clipboard_translater': 'Python',
 'Virtual_environment_research': 'Python',
 'IB_Music_project': 'Jupyter Notebook',
 'Asteroids_pygame_tutorial': 'Python',
 'BI_2021_spring_project': 'Jupyter Notebook',
 'SeqTools': 'Python',
 'BI_Statistics_projects': 'None',
 'BI_Python': 'Jupyter Notebook',
 'MAG_gi_plasmid_analysis': 'Jupyter Notebook'}

In [289]:
get_user_info('krglkvrmn')

{'user_name': 'Roman Kruglikov',
 'organization': 'unknown',
 'city': 'Moscow',
 'followers': '1',
 'repositories': '10'}

In [ ]:
keyreallkeyrealenko/BI_2021_Python

## Finctions END